In [3]:
# 라이브러리 추가
import pandas as pd
import re

In [4]:
# 기사 제목
f = open("D:/GEODATA/bigkinds.txt", 'r', encoding='UTF-8')
lines = f.readlines()
f.close()

In [5]:
# 한나눔 불러오기
from konlpy.tag import Hannanum
hannanum = Hannanum()

In [6]:
# 단어 2차원 리스트
dataset = []
for i in range(len(lines)):
    dataset.append(hannanum.nouns(re.sub('[^가-힣a-zA-Z\s]', '', lines[i])))
dataset[:10]

[['구상나무', '산굴뚝나비', '한라산국립공원', '대표', '생물종'],
 ['한라산', '대표', '생물종', '구상나무', '산굴뚝나비', '선정'],
 ['한라산', '대표', '생물', '구상나무산굴뚝나비', '선정'],
 ['한라산', '생태', '특성', '대표', '생물종', '구상나무', '산굴뚝나비', '선정'],
 ['제주', '한라산', '대표', '생물종', '구상나무', '산굴뚝나비'],
 ['한라산국립공원', '대표', '생물'],
 ['장태산', '문재', '코스', '수', '생물친구들'],
 ['강릉제모습', '순포습지', '순채', '복원', '성', '미미'],
 ['쉬다가고파'],
 ['계룡산', '호반새']]

In [30]:
# 트랜잭션 인코더
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
df[['구상나무', '산굴뚝나비', '한라산국립공원', '대표', '생물종']].head(10)

,구상나무,산굴뚝나비,한라산국립공원,대표,생물종
0,True,True,True,True,True
1,True,True,False,True,True
2,False,False,False,True,False
3,True,True,False,True,True
4,True,True,False,True,True
5,False,False,True,True,False
6,False,False,False,False,False
7,False,False,False,False,False
8,False,False,False,False,False
9,False,False,False,False,False


In [40]:
# 어프라이어리(Apriori)
from mlxtend.frequent_patterns import apriori
# 지지도(support): P(A∩B)
frequent_itemsets = apriori(df, min_support=0.01, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets[(frequent_itemsets['length'] == 2) &
                  (frequent_itemsets['support'] >= 0.01)].sort_values(by='support', ascending=False).head(10)

,support,itemsets,length
74,0.034951,"(선정, 깃대종)",2
73,0.020388,"(대전시, 깃대종)",2
77,0.020388,"(담비, 속리산)",2
82,0.020388,"(속리산, 서식)",2
76,0.016505,"(담비, 서식)",2
85,0.016505,"(확인, 서식)",2
87,0.013592,"(하늘다람쥐, 속리산)",2
72,0.013592,"(대전, 깃대종)",2
71,0.012621,"(계룡산, 호반새)",2
75,0.011650,"(종, 깃대종)",2


In [44]:
# 연관규칙(association rules)
# 신뢰도(confidence): P(B|A) = P(A∩B) / P(A)
# 향상도(lift): P(B|A) / P(B)
from mlxtend.frequent_patterns import association_rules
association_rules(frequent_itemsets, metric="confidence",
                  min_threshold=0.7).sort_values(by='lift', ascending=False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
7,(호반새),(계룡산),0.012621,0.020388,0.012621,1.000000,49.047619,0.012364,inf
1,"(담비, 서식)",(속리산),0.016505,0.026214,0.016505,1.000000,38.148148,0.016072,inf
3,(담비),"(속리산, 서식)",0.022330,0.020388,0.016505,0.739130,36.252588,0.016050,3.755178
2,"(속리산, 서식)",(담비),0.020388,0.022330,0.016505,0.809524,36.252588,0.016050,5.132767
4,(담비),(속리산),0.022330,0.026214,0.020388,0.913043,34.830918,0.019803,11.198544
5,(속리산),(담비),0.026214,0.022330,0.020388,0.777778,34.830918,0.019803,4.399515
0,"(담비, 속리산)",(서식),0.020388,0.040777,0.016505,0.809524,19.852608,0.015673,5.035922
6,(속리산),(서식),0.026214,0.040777,0.020388,0.777778,19.074074,0.019319,4.316505
9,(담비),(서식),0.022330,0.040777,0.016505,0.739130,18.126294,0.015594,3.677023
8,(선정),(깃대종),0.048544,0.126214,0.034951,0.720000,5.704615,0.028825,3.120666


In [55]:
# 신뢰도(confidence): P(B|A) = P(A∩B) / P(A) = P({담비}∩{속리산,서식}) / P({담비})
print('{담비, 속리산, 서식}의 지지도: ' + str(format(0.016505, ".6f")))
print('{담비}→{속리산, 서식} 규칙의 신뢰도: ' + str(format(0.016505 / 0.022330, ".6f")))
print('{담비}→{속리산, 서식} 규칙의 향상도: ' + str(format((0.016505 / 0.022330) / 0.020388, ".6f")))

{담비, 속리산, 서식}의 지지도: 0.016505
{담비}→{속리산, 서식} 규칙의 신뢰도: 0.739140
{담비}→{속리산, 서식} 규칙의 향상도: 36.253687
